# Import

In [67]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import os
import nltk
nltk.download('popular')
from spellchecker import SpellChecker
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from regex import E

import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab



torch.manual_seed(1)

import warnings
warnings.filterwarnings("ignore")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

In [68]:
train_df = pd.read_csv('../data/train_berkeley.csv') 
test_df = pd.read_csv('../data/test_berkeley.csv')

In [69]:
display(train_df)

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...,...
33292,36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
33293,36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
33294,36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
33295,36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [70]:
display(test_df)

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,69,218bfd96197d,0168325D0E24,Being in a car is how we transport ourselves t...,Lead,Adequate
1,70,599e5e16e9ed,0168325D0E24,Well Paris has enforced a driving ban to help ...,Evidence,Adequate
2,71,a398be97f2b7,0168325D0E24,It actually helped having cars stop driving on...,Claim,Adequate
3,72,38a6679de2ed,0168325D0E24,"""Recent studies suggest that Americans are buy...",Evidence,Adequate
4,73,f1cc93a81a32,0168325D0E24,"In the more populated citys, people are using ...",Claim,Effective
...,...,...,...,...,...,...
3463,36725,8d66664512a0,FE3CA06DDCA1,they might want to give the on who is giving t...,Claim,Adequate
3464,36726,4c649f487587,FE3CA06DDCA1,they are give you as much detail as they have ...,Claim,Adequate
3465,36727,b887de306ae7,FE3CA06DDCA1,They will want to give you all the information...,Evidence,Adequate
3466,36728,54851ec259ee,FE3CA06DDCA1,If a person wants good advice they will want t...,Evidence,Adequate


In [71]:
train_df['label'] = 0
for i in range(len(train_df)):
    if train_df.discourse_effectiveness[i] == 'Adequate':
        train_df['label'][i] = 0 
    elif train_df.discourse_effectiveness[i] == 'Effective':    
        train_df['label'][i] = 1
    elif train_df.discourse_effectiveness[i] == 'Ineffective':    
        train_df['label'][i] = 2
        
    

In [72]:
display(train_df)

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,label
0,0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0
1,1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0
2,2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0
3,3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0
4,4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0
...,...,...,...,...,...,...,...
33292,36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,0
33293,36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,0
33294,36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,0
33295,36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,2


In [73]:
test_df['label'] = 0
for i in range(len(test_df)):
    if test_df.discourse_effectiveness[i] == 'Adequate':
        test_df['label'][i] = 0 
    elif test_df.discourse_effectiveness[i] == 'Effective':    
        test_df['label'][i] = 1
    elif test_df.discourse_effectiveness[i] == 'Ineffective':    
        test_df['label'][i] = 2

In [74]:
display(test_df)

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,label
0,69,218bfd96197d,0168325D0E24,Being in a car is how we transport ourselves t...,Lead,Adequate,0
1,70,599e5e16e9ed,0168325D0E24,Well Paris has enforced a driving ban to help ...,Evidence,Adequate,0
2,71,a398be97f2b7,0168325D0E24,It actually helped having cars stop driving on...,Claim,Adequate,0
3,72,38a6679de2ed,0168325D0E24,"""Recent studies suggest that Americans are buy...",Evidence,Adequate,0
4,73,f1cc93a81a32,0168325D0E24,"In the more populated citys, people are using ...",Claim,Effective,1
...,...,...,...,...,...,...,...
3463,36725,8d66664512a0,FE3CA06DDCA1,they might want to give the on who is giving t...,Claim,Adequate,0
3464,36726,4c649f487587,FE3CA06DDCA1,they are give you as much detail as they have ...,Claim,Adequate,0
3465,36727,b887de306ae7,FE3CA06DDCA1,They will want to give you all the information...,Evidence,Adequate,0
3466,36728,54851ec259ee,FE3CA06DDCA1,If a person wants good advice they will want t...,Evidence,Adequate,0


In [75]:

train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])

# converting to a tuple
train_iter = (x for x in train_iter_list) 

train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

In [76]:
test_iter_list = []
for i in range(len(test_df)):
    test_iter_list.append([test_df.label[i], test_df.discourse_text[i]])

# converting to a tuple
test_iter = (x for x in test_iter_list) 

test_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(test_iter)

In [77]:
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter)

In [78]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [79]:

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [80]:
train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])

# converting to a tuple
train_iter = (x for x in train_iter_list) 

train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

dataloader = DataLoader(train_iter, batch_size=500, shuffle=False, collate_fn=collate_batch)
#dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [81]:
# Hyper-parameters 
num_epochs = 10
batch_size = 500
learning_rate = 0.001

In [82]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [83]:
train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])

# converting to a tuple
train_iter = (x for x in train_iter_list) 

train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

In [84]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [85]:
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [86]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(num_epochs, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [87]:
train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])
train_iter = (x for x in train_iter_list) 
train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

test_iter_list = []
for i in range(len(test_df)):
    test_iter_list.append([test_df.label[i], test_df.discourse_text[i]])
test_iter = (x for x in test_iter_list) 
test_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(test_iter)

In [88]:
from torch.utils.data.dataset import random_split

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                             shuffle=True, collate_fn=collate_batch)


for epoch in range(1, num_epochs + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

RuntimeError: Expected i < index_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [ ]:
len(train_dataloader)

64

In [ ]:
for idx, (label, text, offsets) in enumerate(dataloader):
    print(offsets)
    print(text)
    print(label)
    break

tensor([  0,  50, 185, 232, 234, 436, 451, 605])
tensor([  693,  9502,  1953, 11743,     1,  4379,  3723, 77942,   659,  5006,
        35758,   542,   659, 47053, 24140,  5621, 47053,  7711,  2031, 25584,
            4,   942, 11743,   394, 77942,  2594, 67228, 11603,  9799,  5238,
        52757,  3146,  3723, 77942,   659,  5006,   110, 67228,  7711,  3146,
         1439, 25584,  1955, 11129,  1466,   504, 15341,  6533,    86, 67228,
          171, 77942,   302, 30530,  2783,  3860,   272, 34123,    44,   356,
         5621,   120, 77942,    14, 34123,   303, 22362,  3860,   303, 52757,
        16530,     4,    71, 67228,     1,   118,   629,  1273, 22362,  2695,
         4111, 34123,   489,  1955,   108, 30530,  4893, 30530, 77942,    83,
        30530,    31, 47053,  9397,   968,   181, 30530,    15, 47053,  2783,
        24140,  4823, 15341,  5828,   112,  1955, 67228,  1737,    65,   142,
           76, 25584,   118, 13049,    63,    86,  2783, 47053,  1436,   989,
        34123, 